In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# MISSING SRULL embeddings from test
import pandas as pd
import re
from tqdm import tqdm
import time
import nltk
import ast
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize
# import tensorflow as tf
# from transformers import XLMRobertaTokenizer, TFXLMRobertaForSequenceClassification, AutoTokenizer
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import pandas as pd
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, mean_squared_error, log_loss, make_scorer
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss,
    confusion_matrix,
    classification_report
)
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv("/content/drive/MyDrive/IndiaAI_Hackathon/final_train.csv",low_memory=False)
print(train.shape)

test = pd.read_csv("/content/drive/MyDrive/IndiaAI_Hackathon/final_test.csv",low_memory=False)
print(test.shape)

def create_final_category(val):
  if val in ['Sexually Obscene material', 'Sexually Explicit Act', 'Child Pornography CPChild Sexual Abuse Material CSAM',
 'RapeGang Rape RGRSexually Abusive Content']:
    return 0
  elif val in ['Online Financial Fraud', 'Cryptocurrency Crime', 'Online Gambling  Betting']:
    return 1
  elif val in ['Online and Social Media Related Crime',
 'Any Other Cyber Crime',
 'Cyber Attack/ Dependent Crimes',
 'Hacking  Damage to computercomputer system etc',
 'Cyber Terrorism',
 'Ransomware', 'Report Unlawful Content', 'Online Cyber Trafficking']:
    return 2
  else:
    return 3

train['final_category'] = train['category'].apply(create_final_category)
test['final_category'] = test['category'].apply(create_final_category)

print(train['sub_category'].isna().sum())
print(train['sub_category'].isnull().sum())

print(test['sub_category'].isna().sum())
print(test['sub_category'].isnull().sum())

train['final_subcategory'] = train['sub_category'].copy()
train.loc[train['sub_category'].isnull(), 'final_subcategory'] = train[train['sub_category'].isnull()]['category']
print(train['final_subcategory'].isnull().sum(), train.shape)

test['final_subcategory'] = test['sub_category'].copy()
test.loc[test['sub_category'].isnull(), 'final_subcategory'] = test[test['sub_category'].isnull()]['category']
print(test['final_subcategory'].isnull().sum(), test.shape)

train = train[~train['SRLL_msg_updated'].isnull()].reset_index(drop=True)
print(train.shape)
test = test[~test['SRLL_msg_updated'].isnull()].reset_index(drop=True)
print(test.shape)

category_mappings = {'Cyber Bullying  Stalking  Sexting': 'Cyber Bullying/Stalking/Sexting',
 'Fraud CallVishing': 'Fraud Call/Vishing',
 'Online Gambling  Betting': 'Online Gambling/Betting Fraud',
 'Online Job Fraud': 'Online Job Fraud',
 'UPI Related Frauds': 'UPI-Related Frauds',
 'Internet Banking Related Fraud': 'Internet Banking-Related Fraud',
 'RapeGang Rape RGRSexually Abusive Content': 'Rape/Gang Rape-Sexually Abusive Content',
 'Other': 'Any Other Cyber Crime',
 'Profile Hacking Identity Theft': 'Profile Hacking/Identity Theft',
 'DebitCredit Card FraudSim Swap Fraud': 'Debit/Credit Card/Sim Swap Fraud',
 'EWallet Related Fraud': 'E-Wallet Related Frauds',
 'Data Breach/Theft': 'Data Breach/Theft',
 'Denial of Service (DoS)/Distributed Denial of Service (DDOS) attacks': 'Denial of Service (DoS) and Distributed Denial of Service (DDoS) attacks',
 'FakeImpersonating Profile': 'Impersonation Fraud',
 'Cryptocurrency Fraud': 'Cryptocurrency Fraud',
 'Sexually Explicit Act': 'Sale,Publishing and Transmitting Obscene Material/Sexually Explicit Material',
 'Sexually Obscene material': 'Sale,Publishing and Transmitting Obscene Material/Sexually Explicit Material',
 'Malware Attack': 'Malicious code attacks',
 'Business Email CompromiseEmail Takeover': 'Business Email Compromise/Email Takeover',
 'Email Hacking': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'Cheating by Impersonation': 'Impersonation Fraud',
 'Hacking/Defacement': 'Hacking/Website Defacement',
 'Unauthorised AccessData Breach': 'Unauthorized Access/Data Breach',
 'SQL Injection': 'Malicious code attacks',
 'Provocative Speech for unlawful acts': 'Provocative Speech of Unlawful Acts',
 'Ransomware Attack': 'Malicious code attacks',
 'Cyber Terrorism': 'Cyber Terrorism',
 'Child Pornography CPChild Sexual Abuse Material CSAM': 'Child Pornography/Child Sexual Abuse Material(CSAM)',
 'Tampering with computer source documents': 'Tampering with Computer Source Documents',
 'DematDepository Fraud': 'Demat/Depository Fraud',
 'Online Trafficking': 'Online Cyber Trafficking',
 'Online Matrimonial Fraud': 'Online Matrimonial Fraud',
 'Website DefacementHacking': 'Hacking/Website Defacement',
 'Damage to computer computer systems etc': 'Damage to Computer Systems',
 'Impersonating Email': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'EMail Phishing': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'Ransomware': 'Malicious code attacks',
 'Intimidating Email': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'Against Interest of sovereignty or integrity of India': 'Provocative Speech of Unlawful Acts'}

print(test.shape)
test = test[test['final_subcategory'].isin(train['final_subcategory'])].reset_index(drop=True)
print(test.shape)

train['final_subcategory_grouped'] = train['final_subcategory'].replace(category_mappings)
test['final_subcategory_grouped'] = test['final_subcategory'].replace(category_mappings)

(84040, 13)
(28328, 13)
3798
3798
1358
1358
0 (84040, 14)
0 (28328, 14)
(84023, 14)
(28327, 14)
(28327, 14)
(28323, 14)


Grouped Subcategories:

•	"Against Interest of Sovereignty or Integrity of India" & "Provocative Speech of Unlawful Acts" -> Combined into "Provocative Speech of Unlawful Acts"

•	"Email Phishing", "Email Hacking", "Impersonating Email", "Intimidating Email" -> Combined into "Email Hacking/Phishing/Impersonating/Intimidating"
•	"Cheating by Impersonation" & "Fake/Impersonating Profile" -> Combined into "Impersonation Fraud"

•	"Malware Attack", "Ransomware", "Ransomware Attack", "SQL Injection" -> Combined into "Malicious Code Attacks"

•	"Hacking/Defacement" & "Website DefacementHacking" -> Combined into "Hacking/Website Defacement"
•	"Cyber Blackmailing & Threatening" & "Cyber Bullying/Stalking/Sexting" -> Combined into "Cyber Bullying/Stalking/Sexting"


#### Helper

In [ ]:
def post_analysis(df,x_train, x_test, oot, model, overall_feature_bow, x_train_bow, x_test_bow, oot_bow, model_name = 'lr', prediction_feature = 'final_category'):
  from sklearn.metrics import (roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, log_loss, confusion_matrix,classification_report)

  y_test_pred = model.predict(x_test_bow)
  y_test_proba = model.predict_proba(x_test_bow)[:, 1]
  y_train_pred = model.predict(x_train_bow)
  y_train_proba = model.predict_proba(x_train_bow)[:, 1]
  y_overall_pred = model.predict(overall_feature_bow)
  y_overall_proba = model.predict_proba(overall_feature_bow)[:, 1]
  y_oot_pred = model.predict(oot_bow)
  y_oot_proba = model.predict_proba(oot_bow)[:, 1]

  df[model_name+'_model_score'] = model.predict_proba(overall_feature_bow)[:, 1]
  x_train[model_name+'_model_score'] = model.predict_proba(x_train_bow)[:, 1]
  x_test[model_name+'_model_score'] = model.predict_proba(x_test_bow)[:, 1]
  oot[model_name+'_model_score'] = model.predict_proba(oot_bow)[:, 1]

  # Calculate relevant metrics for the training set
  train_metrics = {
      # "AUC": roc_auc_score(y_train, y_train_proba),
      "Accuracy": accuracy_score(y_train, y_train_pred),
      # "Precision": precision_score(y_train, y_test_pred),
      # "Recall": recall_score(y_train, y_test_pred),
      "F1 Score": f1_score(y_train, y_train_pred, average='weighted'),
      # "Log Loss": log_loss(y_train, y_train_proba)
  }

  test_metrics = {
      # "AUC": roc_auc_score(y_test, y_test_proba),
      "Accuracy": accuracy_score(y_test, y_test_pred),
      # "Precision": precision_score(y_test, y_test_pred),
      # "Recall": recall_score(y_test, y_test_pred),
      "F1 Score": f1_score(y_test, y_test_pred, average='weighted'),
      # "Log Loss": log_loss(y_test, y_test_proba)
  }

  overall_metrics = {
      # "AUC": roc_auc_score(df[prediction_feature].tolist(), y_overall_proba),
      "Accuracy": accuracy_score(df[prediction_feature].tolist(), y_overall_pred),
      # "Precision": precision_score(df[prediction_feature].tolist(), y_overall_pred),
      # "Recall": recall_score(df[prediction_feature].tolist(), y_overall_pred),
      "F1 Score": f1_score(df[prediction_feature].tolist(), y_overall_pred, average='weighted'),
      # "Log Loss": log_loss(df[prediction_feature].tolist(), y_overall_proba)
  }

  oot_metrics = {
      # "AUC": roc_auc_score(oot[prediction_feature].tolist(), y_oot_proba),
      "Accuracy": accuracy_score(oot[prediction_feature].tolist(), y_oot_pred),
      # "Precision": precision_score(oot[prediction_feature].tolist(), y_oot_pred),
      # "Recall": recall_score(oot[prediction_feature].tolist(), y_oot_pred),
      "F1 Score": f1_score(oot[prediction_feature].tolist(), y_oot_pred, average='weighted'),
      # "Log Loss": log_loss(oot[prediction_feature].tolist(), y_oot_proba)
  }

  # Create a DataFrame to compare metrics
  metrics_df = pd.DataFrame({
      'Metric': list(train_metrics.keys()),
      'Train': list(train_metrics.values()),
      'Test': list(test_metrics.values()),
      'Overall Data': list(overall_metrics.values()),
      'OOT': list(oot_metrics.values())
  })

  # Display the DataFrame
  return display(metrics_df)

In [4]:
def classwise_scores_and_overall(model, X_test_embed, y_test):

  y_pred = model.predict(X_test_embed)
  # Calculate per-class metrics
  metrics = []
  for i, class_label in enumerate(model.classes_):
      class_accuracy = accuracy_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_f1 = f1_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_recall = recall_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_precision = precision_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))

      metrics.append({
          'Class': class_label,
          'Accuracy': class_accuracy,
          'F1 Score': class_f1,
          'Recall': class_recall,
          'Precision': class_precision
      })

  metrics_df = pd.DataFrame(metrics)

  # Calculate overall metrics
  overall_accuracy = accuracy_score(y_test, y_pred)
  overall_f1 = f1_score(y_test, y_pred, average='weighted')
  overall_recall = recall_score(y_test, y_pred, average='weighted')
  overall_precision = precision_score(y_test, y_pred, average='weighted')

  # Display overall metrics
  overall_metrics = {
      'Overall Accuracy': overall_accuracy,
      'Overall F1 Score': overall_f1,
      'Overall Recall': overall_recall,
      'Overall Precision': overall_precision
  }
  overall_metrics_df = pd.DataFrame([overall_metrics])
  return metrics_df, overall_metrics_df

####**Sub Category Classification**

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the subCategory_updated labels
train['subcategory_label'] = label_encoder.fit_transform(train['final_subcategory_grouped'])
test['subcategory_label'] = label_encoder.fit_transform(test['final_subcategory_grouped'])

# Check the mapping of original labels to encoded labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label mapping:", label_mapping)

Label mapping: {'Any Other Cyber Crime': 0, 'Business Email Compromise/Email Takeover': 1, 'Child Pornography/Child Sexual Abuse Material(CSAM)': 2, 'Cryptocurrency Fraud': 3, 'Cyber Bullying/Stalking/Sexting': 4, 'Cyber Terrorism': 5, 'Damage to Computer Systems': 6, 'Data Breach/Theft': 7, 'Debit/Credit Card/Sim Swap Fraud': 8, 'Demat/Depository Fraud': 9, 'Denial of Service (DoS) and Distributed Denial of Service (DDoS) attacks': 10, 'E-Wallet Related Frauds': 11, 'Email Hacking/Phishing/Impersonating/Intimidating': 12, 'Fraud Call/Vishing': 13, 'Hacking/Website Defacement': 14, 'Impersonation Fraud': 15, 'Internet Banking-Related Fraud': 16, 'Malicious code attacks': 17, 'Online Cyber Trafficking': 18, 'Online Gambling/Betting Fraud': 19, 'Online Job Fraud': 20, 'Online Matrimonial Fraud': 21, 'Profile Hacking/Identity Theft': 22, 'Provocative Speech of Unlawful Acts': 23, 'Rape/Gang Rape-Sexually Abusive Content': 24, 'Sale,Publishing and Transmitting Obscene Material/Sexually Exp

In [6]:
for i in label_mapping.keys():
  print(i)

Any Other Cyber Crime
Business Email Compromise/Email Takeover
Child Pornography/Child Sexual Abuse Material(CSAM)
Cryptocurrency Fraud
Cyber Bullying/Stalking/Sexting
Cyber Terrorism
Damage to Computer Systems
Data Breach/Theft
Debit/Credit Card/Sim Swap Fraud
Demat/Depository Fraud
Denial of Service (DoS) and Distributed Denial of Service (DDoS) attacks
E-Wallet Related Frauds
Email Hacking/Phishing/Impersonating/Intimidating
Fraud Call/Vishing
Hacking/Website Defacement
Impersonation Fraud
Internet Banking-Related Fraud
Malicious code attacks
Online Cyber Trafficking
Online Gambling/Betting Fraud
Online Job Fraud
Online Matrimonial Fraud
Profile Hacking/Identity Theft
Provocative Speech of Unlawful Acts
Rape/Gang Rape-Sexually Abusive Content
Sale,Publishing and Transmitting Obscene Material/Sexually Explicit Material
Tampering with Computer Source Documents
UPI-Related Frauds
Unauthorized Access/Data Breach


In [7]:
train['subcategory_label'] = train['subcategory_label'].apply(lambda x: 24 if x==25 else x)
test['subcategory_label'] = test['subcategory_label'].apply(lambda x: 24 if x==25 else x)

category_0_train = train[train['final_category'] == 0].reset_index(drop=True)
category_1_train = train[train['final_category'] == 1].reset_index(drop=True)
category_2_train = train[train['final_category'] == 2].reset_index(drop=True)

category_0_test = test[test['final_category'] == 0].reset_index(drop=True)
category_1_test = test[test['final_category'] == 1].reset_index(drop=True)
category_2_test = test[test['final_category'] == 2].reset_index(drop=True)


#Category_0 - Women/Child	Related	Crime	- Subcategeories classification

##Logistic regression

### BOW with max features =1000

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import pandas as pd

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
y_oot = test1['subcategory_label'].copy()
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()

logreg = LogisticRegression(max_iter=1000 ,multi_class='ovr')
logreg.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(logreg, X_train_bow, y_train )
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(logreg, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall( logreg,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(logreg, X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall.index = ['train', 'test', 'train_overall', 'oot']

# Display the results
print("Class-wise metrics:")
print(classwise)

print("\nOverall metrics:")
print(overall)


(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)
2319    24
3409    24
742     24
2093    24
1115    24
Name: subcategory_label, dtype: int64
Class-wise metrics:
                 Class  Accuracy  F1 Score    Recall  Precision
train_0              2  0.978566  0.872117  0.793893   0.967442
train_1             24  0.978566  0.988303  0.997291   0.979475
test_0               2  0.917808  0.390625  0.284091   0.625000
test_1              24  0.917808  0.955932  0.982578   0.930693
train_overall_0      2  0.963373  0.770248  0.665714   0.913725
train_overall_1     24  0.963373  0.980100  0.993614   0.966949
oot_0                2  0.927045  0.440678  0.342105   0.619048
oot_1               24  0.927045  0.960978  0.980692   0.942040

Overall metrics:
               Overall Accuracy  Overall F1 Score  Overall Recall  \
train                  0.978566          0.977607        0.978566   
test                   0.917808          0.903512        0.917808   
train_overal

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
train_overall_bow

,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.978566,0.977607,0.978566,0.978368


In [ ]:
test_overall_bow

,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.917808,0.903512,0.917808,0.902346


In [ ]:
train_overall_overall_bow

,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.963373,0.960746,0.963373,0.962041


In [ ]:
oot_overall_bow

,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.927045,0.917268,0.927045,0.914906


### TF-IDF max features=1000

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import pandas as pd

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

logreg = LogisticRegression(max_iter=1000 ,multi_class='ovr')
logreg.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(logreg,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(logreg,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(logreg,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(logreg,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall', 'oot']


(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)
2319    24
3409    24
742     24
2093    24
1115    24
Name: subcategory_label, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
train_overall_tfidf


,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.921996,0.634609,0.593485,0.878619


In [ ]:
test_overall_tfidf


,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.917808,0.595932,0.56702,0.88793


In [ ]:
train_overall_overall_tfidf


,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.920949,0.625294,0.58683,0.88022


In [ ]:
oot_overall_tfidf

,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.929993,0.649357,0.603252,0.879908


## Naive Bayes

###TF-IDF, max features=1000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(nb,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(nb,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(nb,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(nb,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall', 'oot']

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)


###BOW, max features=1000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
y_oot = test1['subcategory_label'].copy()
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()


nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(nb, X_train_bow, y_train )
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(nb, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall( nb,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(nb, X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall.index = ['train', 'test', 'train_overall', 'oot']

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)


## Random Forest

### BOW with max features=1000

In [ ]:
# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)


bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall.index = ['train', 'test', 'train_overall', 'oot']

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)


### TF-IDF with max features=1000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_rf_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
# Adjust the index dynamically based on the row count
classwise_rf_tfidf.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]

overall_rf_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_rf_tfidf.index = ['train', 'test', 'train_overall','oot']

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)


In [ ]:
train_overall_tfidf

,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.996135,0.996132,0.996135,0.996129


In [ ]:
test_overall_tfidf

,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.924131,0.900561,0.924131,0.925658


In [ ]:
train_overall_overall_tfidf

,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.978129,0.977006,0.978129,0.978136


In [ ]:
oot_overall_tfidf

,Overall Accuracy,Overall F1 Score,Overall Recall,Overall Precision
0,0.933677,0.919894,0.933677,0.925097


## XGBoost

### CatBoost

In [ ]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 23.4 MB/s eta 0:00:00


####BOW with max features=1000

In [ ]:
from catboost import CatBoostClassifier

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )

y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall_cat_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_cat_bow.index = ['train', 'test', 'train_overall', 'oot']

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)


In [ ]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.908644,0.907271,0.908300,0.915991
Overall F1 Score,0.865846,0.863160,0.865176,0.875828
Overall Recall,0.908644,0.907271,0.908300,0.915991
Overall Precision,0.916996,0.823140,0.916714,0.839040


#### TF_IDF with max features =1000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()


model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_cat_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_cat_tfidf.index = ['train', 'test', 'train_overall', 'oot']

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)


In [ ]:
overall_cat_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.934996,0.922023,0.931752,0.932940
Overall F1 Score,0.920553,0.901141,0.915843,0.916850
Overall Recall,0.934996,0.922023,0.931752,0.932940
Overall Precision,0.933706,0.913004,0.929037,0.925816


### Gradient Boosting

####BOW with max features=1000

In [ ]:
# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )

y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = GradientBoostingClassifier()
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall_gb_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_gb_bow.index = ['train', 'test', 'train_overall', 'oot']

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)


In [ ]:
overall_gb_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.938510,0.915701,0.932806,0.929993
Overall F1 Score,0.925803,0.896357,0.918545,0.913195
Overall Recall,0.938510,0.915701,0.932806,0.929993
Overall Precision,0.938052,0.897880,0.928610,0.919887


####TF-IDF with max features=1000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()


model = GradientBoostingClassifier()
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_gb_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_gb_tfidf.index = ['train', 'test', 'train_overall', 'oot']

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)


In [ ]:
overall_gb_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.942024,0.918862,0.936232,0.929256
Overall F1 Score,0.931112,0.899748,0.923510,0.915760
Overall Recall,0.942024,0.918862,0.936232,0.929256
Overall Precision,0.941743,0.904163,0.933329,0.917066


In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

# Training Logistic Regression model with BoW
bow_model = SVC(kernel='rbf', gamma='scale', random_state=42)
bow_model.fit(X_train_bow, y_train)

X_oot = bow_vectorizer.transform(test1[feature_name])
X_train_overall = bow_vectorizer.transform(train1[feature_name])
y_oot_pred =  bow_model.predict(X_oot)
y_oot = test1['final_category'].copy()


post_analysis(train1, X_train, X_test, test1, bow_model, X_train_overall,
                X_train_bow, X_test_bow, X_oot, model_name = 'lr',
                prediction_feature = 'final_category')

(84023, 15) (84023, 15)
(28323, 15) (28323, 15)
(63017,) (21006,)
(63017,) (21006,)


#Category_1 - Financial	Fraud	Crimes	- Subcategeories classification



## Logistic regression

### BOW max features=1000

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import pandas as pd

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
y_oot = test1['subcategory_label'].copy()
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()

logreg = LogisticRegression(max_iter=1000 ,multi_class='ovr')
logreg.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(logreg, X_train_bow, y_train )
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(logreg, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall( logreg,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(logreg, X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall.index = ['train', 'test', 'train_overall', 'oot']


(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)
44078    27
26873     8
14242    13
44375    27
45371    11
Name: subcategory_label, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.786608,0.681640,0.760366,0.688152
Overall F1 Score,0.779036,0.665380,0.750904,0.672417
Overall Recall,0.786608,0.681640,0.760366,0.688152
Overall Precision,0.790873,0.665882,0.760793,0.672054


### TF-IDF max features=1000

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import pandas as pd

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

logreg = LogisticRegression(max_iter=1000 ,multi_class='ovr')
logreg.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(logreg,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(logreg,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(logreg,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(logreg,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall', 'oot']


(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)
44078    27
26873     8
14242    13
44375    27
45371    11
Name: subcategory_label, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.713200,0.705367,0.711242,0.704165
Overall F1 Score,0.692514,0.684883,0.690613,0.683734
Overall Recall,0.713200,0.705367,0.711242,0.704165
Overall Precision,0.705056,0.698735,0.703299,0.699334


## Naive Bayes

###TFIDF, max features=1000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(nb,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(nb,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(nb,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(nb,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall', 'oot']

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classifi

In [ ]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.649901,0.644168,0.648468,0.647889
Overall F1 Score,0.626001,0.618689,0.624191,0.624434
Overall Recall,0.649901,0.644168,0.648468,0.647889
Overall Precision,0.642313,0.627156,0.639449,0.633589


###BOW, max features=1000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
y_oot = test1['subcategory_label'].copy()
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()


nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(nb, X_train_bow, y_train )
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(nb, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall( nb,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(nb, X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall_nb_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_nb_bow.index = ['train', 'test', 'train_overall', 'oot']

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)


In [ ]:
overall_nb_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.619238,0.612378,0.617523,0.613559
Overall F1 Score,0.617724,0.610330,0.615880,0.613105
Overall Recall,0.619238,0.612378,0.617523,0.613559
Overall Precision,0.628342,0.621461,0.626622,0.626757


## Random Forest

###BOW with max features=1000

In [ ]:
# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)


bow_vectorizer = CountVectorizer(max_features=2500)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall.index = ['train', 'test', 'train_overall', 'oot']

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)


In [ ]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.989685,0.704062,0.918279,0.706411
Overall F1 Score,0.989681,0.676143,0.917309,0.679363
Overall Recall,0.989685,0.704062,0.918279,0.706411
Overall Precision,0.989716,0.707809,0.920879,0.700172


###TF-IDF with max features=1000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_rf_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
# Adjust the index dynamically based on the row count
classwise_rf_tfidf.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]

overall_rf_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_rf_tfidf.index = ['train', 'test', 'train_overall','oot']

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)


In [ ]:
overall_rf_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.987996,0.705675,0.917415,0.705662
Overall F1 Score,0.987990,0.679421,0.916458,0.679278
Overall Recall,0.987996,0.705675,0.917415,0.705662
Overall Precision,0.988057,0.710090,0.920020,0.694141


## XGBoost

### CatBoost

####BOW with max features=1000

In [ ]:
from catboost import CatBoostClassifier

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )

y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall_cat_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_cat_bow.index = ['train', 'test', 'train_overall', 'oot']

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [ ]:
overall_cat_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.684507,0.681179,0.683675,0.690110
Overall F1 Score,0.655180,0.651151,0.654178,0.661481
Overall Recall,0.684507,0.681179,0.683675,0.690110
Overall Precision,0.670059,0.666886,0.669246,0.675933


####TF-IDF with max features=1000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()


model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_cat_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_cat_tfidf.index = ['train', 'test', 'train_overall', 'oot']

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [ ]:
overall_cat_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.684532,0.684097,0.684424,0.689592
Overall F1 Score,0.654625,0.654119,0.654504,0.660084
Overall Recall,0.684532,0.684097,0.684424,0.689592
Overall Precision,0.669832,0.671432,0.670219,0.675794


### Gradient Boosting

####BOW with max features=1000

In [ ]:
# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )

y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = GradientBoostingClassifier()
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall_gb_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_gb_bow.index = ['train', 'test', 'train_overall', 'oot']

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)


In [ ]:
overall_gb_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.721390,0.697151,0.715331,0.699326
Overall F1 Score,0.705396,0.676176,0.698150,0.678259
Overall Recall,0.721390,0.697151,0.715331,0.699326
Overall Precision,0.727240,0.688604,0.718127,0.689636


####TF-IDF with max features=10000

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()


model = GradientBoostingClassifier()
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_gb_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_gb_tfidf.index = ['train', 'test', 'train_overall', 'oot']

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)


In [ ]:
overall_gb_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.734931,0.693696,0.724622,0.698174
Overall F1 Score,0.720701,0.673440,0.709259,0.678543
Overall Recall,0.734931,0.693696,0.724622,0.698174
Overall Precision,0.739948,0.686173,0.727866,0.689344


#Category_2 - Other	Cyber	Crime	- Subcategeories classification



##Logistic regression

### BOW max features=1000

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import pandas as pd

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

bow_vectorizer = CountVectorizer(max_features=5000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
y_oot = test1['subcategory_label'].copy()
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()

logreg = LogisticRegression(max_iter=1000 ,multi_class='ovr')
logreg.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(logreg, X_train_bow, y_train )
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(logreg, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall( logreg,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(logreg, X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall.index = ['train', 'test', 'train_overall', 'oot']


(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)
24892     0
10485     0
2770      0
14391     0
12554    15
Name: subcategory_label, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: Undefin

In [10]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.748270,0.556298,0.700277,0.548256
Overall F1 Score,0.722623,0.514141,0.671478,0.505977
Overall Recall,0.748270,0.556298,0.700277,0.548256
Overall Precision,0.761114,0.506175,0.707174,0.492460


### TF-IDF max features=1000

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import pandas as pd

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

logreg = LogisticRegression(max_iter=1000 ,multi_class='ovr')
logreg.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(logreg,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(logreg,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(logreg,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(logreg,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall', 'oot']


(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)
24892     0
10485     0
2770      0
14391     0
12554    15
Name: subcategory_label, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: Undefin

In [13]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.609990,0.582030,0.603000,0.575950
Overall F1 Score,0.558216,0.526305,0.550262,0.520780
Overall Recall,0.609990,0.582030,0.603000,0.575950
Overall Precision,0.562494,0.520346,0.552301,0.519996


## Naive Bayes

###TF-IDF, max features=1000

In [14]:
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(nb,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(nb,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(nb,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(nb,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall', 'oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [15]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.583831,0.577054,0.582137,0.563561
Overall F1 Score,0.518376,0.506321,0.515408,0.493439
Overall Recall,0.583831,0.577054,0.582137,0.563561
Overall Precision,0.546470,0.512388,0.539312,0.519286


###BOW, max features=1000

In [16]:
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
y_oot = test1['subcategory_label'].copy()
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()


nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(nb, X_train_bow, y_train )
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(nb, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall( nb,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(nb, X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall_nb_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_nb_bow.index = ['train', 'test', 'train_overall', 'oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classifi

In [17]:
overall_nb_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.557104,0.543076,0.553597,0.536491
Overall F1 Score,0.534405,0.517433,0.530153,0.513069
Overall Recall,0.557104,0.543076,0.553597,0.536491
Overall Precision,0.546370,0.522456,0.540416,0.518534


## Random Forest

###BOW with max features=1000

In [18]:
# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)


bow_vectorizer = CountVectorizer(max_features=2500)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall.index = ['train', 'test', 'train_overall', 'oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classifi

In [20]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.996920,0.574637,0.891349,0.570432
Overall F1 Score,0.996918,0.521447,0.889903,0.517537
Overall Recall,0.996920,0.574637,0.891349,0.570432
Overall Precision,0.996933,0.540651,0.896395,0.547823


###TF-IDF with max features=1000

In [21]:
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)
X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_rf_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
# Adjust the index dynamically based on the row count
classwise_rf_tfidf.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]

overall_rf_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_rf_tfidf.index = ['train', 'test', 'train_overall','oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [22]:
overall_rf_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.995214,0.562411,0.887013,0.563352
Overall F1 Score,0.995211,0.515461,0.885623,0.517733
Overall Recall,0.995214,0.562411,0.887013,0.563352
Overall Precision,0.995252,0.531505,0.891398,0.542258


## XGBoost

### CatBoost

####BOW with max features=1000

In [34]:
from catboost import CatBoostClassifier

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )

y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall_cat_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_cat_bow.index = ['train', 'test', 'train_overall', 'oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [35]:
overall_cat_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.573927,0.563691,0.571368,0.56075
Overall F1 Score,0.515534,0.505622,0.513062,0.50271
Overall Recall,0.573927,0.563691,0.571368,0.56075
Overall Precision,0.515776,0.499972,0.511708,0.50352


####TF-IDF with max features=1000

In [36]:
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()


model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_cat_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_cat_tfidf.index = ['train', 'test', 'train_overall', 'oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [37]:
overall_cat_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.575348,0.566534,0.573145,0.560541
Overall F1 Score,0.516456,0.506913,0.514097,0.501251
Overall Recall,0.575348,0.566534,0.573145,0.560541
Overall Precision,0.519066,0.503382,0.515151,0.502865


### Gradient Boosting

####BOW with max features=1000

In [29]:
# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )

y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = GradientBoostingClassifier()
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall_gb_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_gb_bow.index = ['train', 'test', 'train_overall', 'oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)


In [30]:
overall_gb_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.649559,0.563691,0.628092,0.558980
Overall F1 Score,0.615116,0.516391,0.590942,0.513655
Overall Recall,0.649559,0.563691,0.628092,0.558980
Overall Precision,0.681147,0.519148,0.642238,0.520101


####TF-IDF with max features=1000

In [31]:
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()


model = GradientBoostingClassifier()
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_gb_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_gb_tfidf.index = ['train', 'test', 'train_overall', 'oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)


In [32]:
overall_gb_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.690172,0.552744,0.655815,0.553358
Overall F1 Score,0.670172,0.514197,0.632046,0.514701
Overall Recall,0.690172,0.552744,0.655815,0.553358
Overall Precision,0.722080,0.518679,0.673743,0.522313


# Hyperparameter tuning

##Category_2 classification

###logistic regression

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import pandas as pd
import time

feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

# Hyperparameter tuning using GridSearchCV
s = time.time()
lr = LogisticRegression(multi_class='multinomial', max_iter=1000)
param_grid = {
    'C': [0.1, 1, 10],                          # Regularization strength
    'penalty': ['l2'],                          # 'l2' penalty works with 'lbfgs' and 'newton-cg' solvers for multinomial
    'solver': ['lbfgs', 'newton-cg', 'saga'],   # Solvers that support 'multinomial' with L2 penalty
    'class_weight': [None, 'balanced'],         # Class weight options
}

grid_search = GridSearchCV(
    estimator=lr,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# Fit grid search to the data
grid_search.fit(X_train_tfidf, y_train)

# Best parameters found by GridSearch
print(f"Best parameters found: {grid_search.best_params_}")
e = time.time()
print(f"Time taken for GridSearch: {(e-s)/60:.2f} mins")

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)
y_pred_proba = best_model.predict_proba(X_test_tfidf)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(pd.get_dummies(y_test), y_pred_proba, multi_class='ovr'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)
24892     0
10485     0
2770      0
14391     0
12554    15
Name: subcategory_label, dtype: int64
Fitting 5 folds for each of 18 candidates, totalling 90 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Best parameters found: {'C': 0.1, 'class_weight': None, 'penalty': 'l2', 'solver': 'lbfgs'}
Time taken for GridSearch: 2.57 mins
Accuracy: 0.570941143019619
ROC AUC: 0.8598616555605151
F1 Score: 0.4983936542350128


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import pandas as pd
import time
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE

feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2), sublinear_tf=True)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Handle Imbalance with SMOTE
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_tfidf, y_train)

# Hyperparameter Tuning
s = time.time()
lr = LogisticRegression(multi_class='multinomial', max_iter=1000)
param_grid = {
    'C': [0.5, 1, 2],
    'solver': ['lbfgs', 'newton-cg'],
    'class_weight': [None, 'balanced'],
}

grid_search = GridSearchCV(
    estimator=lr,
    param_grid=param_grid,
    scoring='f1_weighted',  # Focus on F1 score
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train_resampled, y_train_resampled)

# Evaluate Best Model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)
y_pred_proba = best_model.predict_proba(X_test_tfidf)

print(f"Best Parameters: {grid_search.best_params_}")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("ROC AUC:", roc_auc_score(pd.get_dummies(y_test), y_pred_proba, multi_class='ovr'))
e = time.time()
print(f"Time taken for tuning: {(e-s)/60:.2f} mins")

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)
24892     0
10485     0
2770      0
14391     0
12554    15
Name: subcategory_label, dtype: int64
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Best Parameters: {'C': 2, 'class_weight': 'balanced', 'solver': 'newton-cg'}
Accuracy: 0.4145578618140461
F1 Score: 0.44992597707234283
ROC AUC: 0.8556443949959835
Time taken for tuning: 6.55 mins


###without smote

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import pandas as pd
import time
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE

feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Exploring more values of C
    'penalty': ['l2'],
    'solver': ['lbfgs', 'newton-cg', 'saga', 'liblinear'],  # Adding 'liblinear' solver
    'class_weight': [None, 'balanced'],
}
grid_search = GridSearchCV(
    estimator=lr,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X_train_tfidf, y_train)


# Evaluate Best Model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)
y_pred_proba = best_model.predict_proba(X_test_tfidf)

print(f"Best Parameters: {grid_search.best_params_}")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("ROC AUC:", roc_auc_score(pd.get_dummies(y_test), y_pred_proba, multi_class='ovr'))

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)
24892     0
10485     0
2770      0
14391     0
12554    15
Name: subcategory_label, dtype: int64
Fitting 5 folds for each of 40 candidates, totalling 200 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
50 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1267, in fit
    multi_class = _check_multi_class(multi_class, solver, len(self.classes_

Best Parameters: {'C': 1, 'class_weight': None, 'penalty': 'l2', 'solver': 'newton-cg'}
Accuracy: 0.5837361387546204
F1 Score: 0.5330881486616466
ROC AUC: 0.87369040359641


###cat boosting

In [ ]:
import time
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Assuming category_2_train and category_2_test are pre-loaded DataFrames

# Feature name to be used for training
feature_name = 'SRLL_msg_updated'

# Data Preparation: Remove rows with null values in the specified feature column
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(f"Train Shape: {train1.shape}, Full Train Shape: {category_2_train.shape}")
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(f"Test Shape: {test1.shape}, Full Test Shape: {category_2_test.shape}")

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'],
    test_size=0.25,
    random_state=42,
    stratify=train1['subcategory_label']
)
print(f"X_train Shape: {X_train.shape}, X_test Shape: {X_test.shape}")
print(f"y_train Shape: {y_train.shape}, y_test Shape: {y_test.shape}")

# Apply TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

# Initialize CatBoost Classifier
catboost_model = CatBoostClassifier(verbose=0, random_seed=42)

# Define the hyperparameter grid for tuning
param_grid = {
    'iterations': [50, 100, 200],          # Number of boosting iterations
    'learning_rate': [0.01, 0.05, 0.1],    # Learning rate
    'depth': [4, 6, 8],                    # Depth of the trees
    'l2_leaf_reg': [1, 3, 5],               # L2 regularization coefficient
    'border_count': [32, 50, 64],           # Number of bins for numerical features
    'random_seed': [42],                   # Random seed for reproducibility
}

# Initialize GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(
    estimator=catboost_model,
    param_grid=param_grid,
    scoring='accuracy',                     # You can use 'roc_auc' or 'f1_weighted' depending on your needs
    cv=5,                                   # Cross-validation with 5 folds
    verbose=1,
    n_jobs=-1
)

# Track the time taken for Grid Search
s = time.time()

# Perform the grid search on the training data
grid_search.fit(X_train_tfidf, y_train)

# Track end time
e = time.time()
print(f"Time taken for grid search: {(e - s) / 60} minutes")

# Best parameters found by GridSearchCV
print(f"Best parameters found: {grid_search.best_params_}")

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_

# Predict on test data
y_pred = best_model.predict(X_test_tfidf)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Test F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

# Optionally, evaluate on the out-of-time (OOT) data
y_oot_pred = best_model.predict(X_oot_tfidf)
print(f"OOT Accuracy: {accuracy_score(y_oot, y_oot_pred)}")
print(f"OOT F1 Score: {f1_score(y_oot, y_oot_pred, average='weighted')}")


In [17]:
import time
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Assuming category_2_train and category_2_test are pre-loaded DataFrames

# Feature name to be used for training
feature_name = 'SRLL_msg_updated'

# Data Preparation: Remove rows with null values in the specified feature column
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(f"Train Shape: {train1.shape}, Full Train Shape: {category_2_train.shape}")
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(f"Test Shape: {test1.shape}, Full Test Shape: {category_2_test.shape}")

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'],
    test_size=0.25,
    random_state=42,
    stratify=train1['subcategory_label']
)
print(f"X_train Shape: {X_train.shape}, X_test Shape: {X_test.shape}")
print(f"y_train Shape: {y_train.shape}, y_test Shape: {y_test.shape}")

# Apply TF-IDF Vectorization with bigrams and trigrams, and tune max_df and min_df
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 3),  # Use unigrams, bigrams, and trigrams
    max_df=0.95,         # Ignore terms that appear in more than 95% of the documents
    min_df=2,            # Only consider terms that appear in at least 2 documents
    stop_words='english' # Remove common stopwords
)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

# Initialize Naive Bayes Classifier
nb_model = MultinomialNB()

# Define the hyperparameter grid for tuning
param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1, 2],  # Smoothing parameter for Naive Bayes
}

# Initialize GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(
    estimator=nb_model,
    param_grid=param_grid,
    scoring='accuracy',                     # You can use 'roc_auc' or 'f1_weighted' depending on your needs
    cv=5,                                   # Cross-validation with 5 folds
    verbose=1,
    n_jobs=-1
)

# Track the time taken for Grid Search
s = time.time()

# Perform the grid search on the training data
grid_search.fit(X_train_tfidf, y_train)

# Track end time
e = time.time()
print(f"Time taken for grid search: {(e - s) / 60} minutes")

# Best parameters found by GridSearchCV
print(f"Best parameters found: {grid_search.best_params_}")

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_

# Predict on test data
y_pred = best_model.predict(X_test_tfidf)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Test F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

# Optionally, evaluate on the out-of-time (OOT) data
y_oot_pred = best_model.predict(X_oot_tfidf)
print(f"OOT Accuracy: {accuracy_score(y_oot, y_oot_pred)}")
print(f"OOT F1 Score: {f1_score(y_oot, y_oot_pred, average='weighted')}")


Train Shape: (28136, 16), Full Train Shape: (28136, 16)
Test Shape: (9605, 16), Full Test Shape: (9605, 16)
X_train Shape: (21102,), X_test Shape: (7034,)
y_train Shape: (21102,), y_test Shape: (7034,)
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Time taken for grid search: 0.004560840129852295 minutes
Best parameters found: {'alpha': 0.5}
Test Accuracy: 0.5735001421666193
Test F1 Score: 0.5099241584221998
OOT Accuracy: 0.5623112961998958
OOT F1 Score: 0.4969610680500643
